In [1]:
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.metrics import (
    accuracy_score,
    f1_score,
    precision_score,
    recall_score,
)

In [2]:
husk_res = Path("results") / "husky"
vulp_res = Path("results") / "vulpi"

# hbox : metrics/husky/CNN-1700-nohamming.dat
# hham : metrics/husky/CNN-1700-hamming.dat
# vbox : metrics/vulpi/CNN-1700-optim6.dat
# vham : metrics/vulpi/CNN-1700-hamming.dat

paths = {
    "husky": {
        "boxcar": husk_res / "results_CNN_nohamming_mw_1.7.npy",
        "hamming": husk_res / "results_CNN_hamming_mw_1.7.npy",
    },
    "vulpi": {
        "boxcar": vulp_res / "results_CNN_optim6_mw_1.7.npy",
        "hamming": vulp_res / "results_CNN_hamming_mw_1.7.npy",
    },
}

values = {}
values["husky"] = {
    "boxc": np.load(paths["husky"]["boxcar"], allow_pickle=True).item(),
    "hamm": np.load(paths["husky"]["hamming"], allow_pickle=True).item(),
}
values["vulpi"] = {
    "boxc": np.load(paths["vulpi"]["boxcar"], allow_pickle=True).item(),
    "hamm": np.load(paths["vulpi"]["hamming"], allow_pickle=True).item(),
}

In [7]:
def convert_to_str(x, terrains):
    if x.dtype.type is np.int64:
        return terrains[x]
    return x


def compute_metrics(clf_values):
    terrains = np.array([t for t in clf_values["terrains"] if t != "MIXED"])
    inv_terrains = {t: i for i, t in enumerate(terrains)}
    ypred = convert_to_str(clf_values["pred"], terrains)
    ytest = convert_to_str(clf_values["true"], terrains)
    metrics = pd.Series(
        {
            "accuracy": accuracy_score(
                ytest,
                ypred,
            ),
            "precision": precision_score(
                ytest,
                ypred,
                labels=terrains,
                average=None,
            ),
            "recall": recall_score(
                ytest,
                ypred,
                labels=terrains,
                average=None,
            ),
            "f1-score": f1_score(
                ytest,
                ypred,
                labels=terrains,
                average=None,
            ),
        }
    )
    return metrics

In [8]:
df = pd.DataFrame(
    {
        "vbox": compute_metrics(values["vulpi"]["boxc"]),
        "vham": compute_metrics(values["vulpi"]["hamm"]),
        "hbox": compute_metrics(values["husky"]["boxc"]),
        "hham": compute_metrics(values["husky"]["hamm"]),
    }
)
df


,vbox,vham,hbox,hham
accuracy,0.934804,0.941176,0.937451,0.939623
precision,"[0.9780439121756487, 0.9152542372881356, 0.971...","[0.9921104536489151, 0.9239543726235742, 0.969...","[0.9335154826958105, 0.96158290005777, 0.97579...","[0.929803328290469, 0.9729099912612875, 0.9724..."
recall,"[0.928030303030303, 0.9204545454545454, 0.9937...","[0.9526515151515151, 0.9204545454545454, 0.989...","[0.8394758394758395, 0.9837470449172577, 0.985...","[0.838929838929839, 0.9869976359338062, 0.9811..."
f1-score,"[0.9523809523809523, 0.9178470254957507, 0.982...","[0.9719806763285024, 0.9222011385199241, 0.979...","[0.8840017248814144, 0.9725387087350278, 0.980...","[0.8820321469575201, 0.9799031832184245, 0.976..."


In [12]:
vulp_diff = df.vham - df.vbox
vulp_diff.accuracy, vulp_diff.precision

(array([ 0.01406654,  0.00870014, -0.00209901,  0.00386847]),
 0.0063725490196078205)

In [16]:
hamm_diff = df.hham - df.hbox
hamm_diff.accuracy, hamm_diff.precision

(0.0021722178901637523,
 array([-0.00371215,  0.01132709, -0.00332396,  0.00976874, -0.00208042]))